In [9]:
get_mars_weather <- function(
    api_key = "DEMO_KEY",
    version = 1.0
) {
    if (!require(httr)) install.packages("httr")
    if (!require(jsonlite)) install.packages("jsonlite")
    if (!require(tidyr)) install.packages("tidyr")
    if (!require(dplyr)) install.packages("dplyr")
    library(httr)
    library(jsonlite)
    library(tidyr)
    library(dplyr)

    feedtype = "json"
    base_url <- "https://api.nasa.gov/insight_weather/"
    
    if (tolower(feedtype) != "json") {
        warning("The API only supports 'json' as the feedtype. Setting feedtype to 'json'.")
        feedtype <- "json"
    }
    query_params <- list(
        ver = version,
        feedtype = feedtype,
        api_key = api_key
    )
    response <- GET(
        url = base_url,
        query = query_params
    )
    if (status_code(response) != 200) {
        stop("Error: Failed to retrieve data. HTTP Status Code: ", status_code(response), "\n",
            content(response, "text"))
    }
    weather_data <- fromJSON(content(response, "text", encoding = "UTF-8"))
    sol_keys <- names(weather_data)[!names(weather_data) %in% c("sol_hours_with_data", "validity_checks")]
  
    if (length(sol_keys) == 0) {
        stop("No sol data available in the response.")
    }
  
    sol_list <- list()
    for (sol in sol_keys) {
        sol_data <- weather_data[[sol]]
        sol_row <- list(sol = as.numeric(sol))
        
        if ("AT" %in% names(sol_data)) {
            sol_row$AT_avg <- sol_data$AT$av
            sol_row$AT_min <- sol_data$AT$mn
            sol_row$AT_max <- sol_data$AT$mx
            sol_row$AT_ct <- sol_data$AT$ct
        } else {
            sol_row$AT_avg <- NA
            sol_row$AT_min <- NA
            sol_row$AT_max <- NA
            sol_row$AT_ct <- NA
        }
        if ("PRE" %in% names(sol_data)) {
            sol_row$PRE_avg <- sol_data$PRE$av
            sol_row$PRE_min <- sol_data$PRE$mn
            sol_row$PRE_max <- sol_data$PRE$mx
            sol_row$PRE_ct <- sol_data$PRE$ct
        } else {
            sol_row$PRE_avg <- NA
            sol_row$PRE_min <- NA
            sol_row$PRE_max <- NA
            sol_row$PRE_ct <- NA
        }
        if ("HWS" %in% names(sol_data)) {
            sol_row$HWS_avg <- sol_data$HWS$av
            sol_row$HWS_min <- sol_data$HWS$mn
            sol_row$HWS_max <- sol_data$HWS$mx
            sol_row$HWS_ct <- sol_data$HWS$ct
        } else {
            sol_row$HWS_avg <- NA
            sol_row$HWS_min <- NA
            sol_row$HWS_max <- NA
            sol_row$HWS_ct <- NA
        }
        
        if ("WD" %in% names(sol_data) && "most_common" %in% names(sol_data$WD)) {
            wd_most_common <- sol_data$WD$most_common
            sol_row$WD_most_common_compass_degrees <- wd_most_common$compass_degrees
            sol_row$WD_most_common_compass_point <- wd_most_common$compass_point
            sol_row$WD_most_common_ct <- wd_most_common$ct
            } else {
            sol_row$WD_most_common_compass_degrees <- NA
            sol_row$WD_most_common_compass_point <- NA
            sol_row$WD_most_common_ct <- NA
        }

        sol_row$First_UTC <- if ("First_UTC" %in% names(sol_data)) sol_data$First_UTC else NA
        sol_row$Last_UTC <- if ("Last_UTC" %in% names(sol_data)) sol_data$Last_UTC else NA
        sol_row$Season <- if ("Season" %in% names(sol_data)) sol_data$Season else NA
        sol_list[[sol]] <- sol_row
    }
  

    weather_df <- bind_rows(sol_list)
    return(weather_df)
}

In [10]:
api_key <- "XFsDPHBjdABhXfquwqnJSfhwEY4rCZI2ev2NDU0K"

weather_df <- get_mars_weather(
    api_key = api_key,
    version = 1.0
)

print("Weather data as a data frame:")
print(head(weather_df))

Loading required package: tidyr

Loading required package: dplyr


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message in get_mars_weather(api_key = api_key, version = 1):
"NAs introduced by coercion"


[1] "Weather data as a data frame:"
# A tibble: 6 x 19
    sol AT_avg AT_min AT_max  AT_ct PRE_avg PRE_min PRE_max PRE_ct HWS_avg
  <dbl>  <dbl>  <dbl>  <dbl>  <int>   <dbl>   <dbl>   <dbl>  <int>   <dbl>
1   675  -62.3  -96.9 -15.9  177556    751.    722.    769. 887776    7.23
2   676  -62.8  -96.9 -16.5  177554    749.    722.    767. 887777    8.53
3   677  -63.1  -97.2 -16.9  177556    749.    721.    767. 887776    7.89
4   678  -62.6  -97.7  -9.06 177556    744.    718.    760. 887777    5.25
5   679  -62.6  -96.6 -11.6  177554    745.    719.    763. 887776    5.56
6   680  -61.8  -96.8 -15.3  168902    744.    717.    764. 887767    6.52
# i 9 more variables: HWS_min <dbl>, HWS_max <dbl>, HWS_ct <int>,
#   WD_most_common_compass_degrees <dbl>, WD_most_common_compass_point <chr>,
#   WD_most_common_ct <int>, First_UTC <chr>, Last_UTC <chr>, Season <chr>
